In [ ]:
%matplotlib inline
import numpy as np
import imageio
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import seaborn as sns

sns.set_style("dark")
plt.style.use("dark_background")

import torch
from pytorch3d.utils import ico_sphere

import utils
import models

from IPython import display

device = torch.device("cuda:0")

In [ ]:
folder_path = "data/"
model_path = "trained_models/model_1588703439/model_0.0519.pth"
plot_sample_ix = 0
points_to_sample = 1000
sphere_level = 3
sphere_level_verts = {2: 162, 3: 642, 4: 2562}
model_names = utils.get_model_names(folder_path, randomize=False)
_, _, test_x, test_y = utils.read_data(model_names, folder_path, points_to_sample, sphere_level_verts[sphere_level])

In [ ]:
model = models.PointNetCls(sphere_level_verts[sphere_level]).to(device)
model.load_state_dict(torch.load(model_path))
model.eval()

In [ ]:
new_src_mesh = ico_sphere(sphere_level).offset_verts(test_y[plot_sample_ix])
image_ref = utils.render_obj(new_src_mesh.verts_packed(), new_src_mesh.faces_packed(), distance=4, elevation=10.0, azimuth=60.0)
fig, ax = plt.subplots(figsize=(5, 5))
plt.axis('off')
plt.imshow(image_ref.squeeze())

In [ ]:
pred_deform_verts = model(torch.from_numpy(test_x[plot_sample_ix]).to(device).unsqueeze(0))
new_src_mesh = ico_sphere(4).offset_verts(pred_deform_verts.detach().cpu().squeeze())
image_ref = utils.render_obj(new_src_mesh.verts_packed(), new_src_mesh.faces_packed(), distance=2, elevation=10.0, azimuth=60.0)
fig, ax = plt.subplots(figsize=(4, 4))
plt.axis('off')
plt.imshow(image_ref.squeeze())

In [ ]:
mesh = utils.load_mesh("./meshes/SM_Env_TreeBirch_01.obj")
image_ref = utils.render_obj(mesh.verts_packed(), mesh.faces_packed(), distance=2, elevation=0.0, azimuth=0.0)
fig, ax = plt.subplots(figsize=(4, 4))
plt.axis('off')
fig.patch.set_facecolor('black')
plt.imshow(image_ref.squeeze())

In [ ]:
mesh = utils.load_mesh("./meshes/SM_Icon_Balloon_01.obj")
anim_file = utils.animate_pointcloud(mesh, f"outputs/SM_Icon_Balloon_01.gif", points_to_sample, restore_anim=False)
display.Image(filename=anim_file)

In [ ]:
anim_file = f"outputs/treebirch_mesh.gif"
frames = []
for plot_i in range(24):
    image_ref = utils.render_obj(mesh.verts_packed(), mesh.faces_packed(), distance=2, elevation=0.0, azimuth=-360*(plot_i/24))
    fig, ax = plt.subplots(figsize=(5, 5))
    canvas = FigureCanvas(fig)
    plt.axis('off')
    plt.imshow(image_ref.squeeze())
    plt.close()
    canvas.draw()
    s, (width, height) = canvas.print_to_buffer()
    frames.append(np.frombuffer(s, np.uint8).reshape((height, width, 4)))
imageio.mimsave(anim_file, frames, 'GIF', fps=8)

In [ ]:
display.Image(filename=anim_file)

In [ ]:
image_ref = utils.render_obj(mesh.verts_packed(), mesh.faces_packed(), distance=2, elevation=0.0, azimuth=0.0)
fig, ax = plt.subplots(figsize=(5, 5))
plt.axis('off')
fig.patch.set_facecolor('black')
plt.imshow(image_ref.squeeze())